## <b>Preprocessing</b>

In [1]:
!pip install autokeras

# Imports the required dependencies
from sklearn.model_selection import train_test_split         # Enables the splitting of the input data into training and test datasets
from sklearn.preprocessing import StandardScaler             # Enables the scaling of the training and test data sets to minimise the risks associated with raw numerical data e.g. outlier impact, different units, skewed distribution
import pandas as pd                                          # Enables the ability to work with DataFrames
import tensorflow as tf                                      # Enables the deploymentof machine learning models
import keras_tuner as kt                                     # Enables auto optimisation of hyperparameters
from tensorflow.keras.callbacks import ModelCheckpoint       # Enables interval saving of the model's weights
from google.colab import files                               # Enables files to be exported

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.5/573.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 944.9/944.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 19.8 MB/s eta 0:00:00


In [2]:
# Imports and reads the charity_data.csv
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# Displays the first five rows within the newly created DataFrame
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


Target variable:
- IS_SUCCESSFUL

Feature variables for model:
- APPLICATION_TYPE
- AFFILIATION
- CLASSIFICATION
- USE_CASE
- ORGANIZATION
- STATUS
- INCOME_AMT
- SPECIAL_CONSIDERATIONS
- ASK_AMT

In [3]:
# Drops the non-beneficial ID columns, 'EIN' and 'NAME' as they won't provide the machine learning model with meaningful insights
application_df = application_df.drop(["EIN", "NAME"], 1)

# Displays the first five rows within the DataFrame to check that the non-beneficial columns have been dropped
application_df.head()

<ipython-input-3-8f8ce177d0e0>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(["EIN", "NAME"], 1)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determines the number of unique values in each column
# Any categorical variables which contain 10 or more unique values will apply the bucketing strategy to optimise memory resourcing
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# The APPLICATION_TYPE column had > 10 unique values, therefore it's value counts will be observed to help inform on which binning approach to take
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Any application type which represent less than 1% of the total number of applications will be binned into a category called 'Other'

# APPLICATION_TYPE value counts < 343 (1% of the total applications)
application_types_below_threshold = application_df["APPLICATION_TYPE"].value_counts() < 343

# A list of the application types to be replaced
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_types_below_threshold].index.tolist()

# Loops through each application type to be replaced and replaces it's value to "Other" within the application DataFrame.
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Reviews the value counts in the APPLICATION_TYPE column to verify that the requested application types are now binned together within the 'Other' category
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# The CLASSIFICATION column also had > 10 unique values, therefore it's value counts will be observed to help inform on which binning approach to take
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# CLASSIFICATION value counts > 1
classifications_above_one = application_df["CLASSIFICATION"].value_counts() > 1
application_df["CLASSIFICATION"].value_counts()[classifications_above_one ]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Any classification which represents less than 1% of the total number of applications will be binned into a category called 'Other'

# CLASSIFICATION value counts < 343 (1% of the total applications)
classifications_below_threshold = application_df["CLASSIFICATION"].value_counts() < 343

# A list of the classifications to be replaced
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[classifications_below_threshold].index.tolist()

# Loops through each classification to be replaced and replaces it's value to "Other" within the application DataFrame.
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Reviews the value counts in the CLASSIFICATION column to verify that the requested clasifications are now binned together within the 'Other' category
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [10]:
# A list of all the columns in the application DataFrame with an object data type
application_categories = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Converts the categorical data to numerical values with `pd.get_dummies`
encode_df = pd.get_dummies(application_df[application_categories])

encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [11]:
# Merges the encoded features to the application DataFrame
application_df = application_df.merge(encode_df, left_index=True, right_index=True)

# Drops the original categories from the application DataFrame
application_df = application_df.drop(application_categories, 1)

# Displays the first five rows within the amended application DataFrame
application_df.head()

<ipython-input-11-c7a27cd05834>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(application_categories, 1)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [12]:
# Splits the preprocessed data into features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"], 1).values

# Splits the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

<ipython-input-12-32b9bd2fed4b>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df.drop(["IS_SUCCESSFUL"], 1).values


In [13]:
# Creates a StandardScaler instance
scaler = StandardScaler()

# Fits the StandardScaler to the training features
X_scaler = scaler.fit(X_train)

# Scales the training and testing features
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## <b>Compile, Train and Evaluate the Model</b>

In [14]:
# Creates a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    number_input_features = len(X_train[0])

    # Allows kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])

    # Allows kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=number_input_features))

    # Allows kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))

    # Output layer
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compiles the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

# Sets up the hyperparameter tuning process
tuner = kt.Hyperband(
                    create_model,
                    objective="val_accuracy",
                    max_epochs=20,
                    hyperband_iterations=2
                    )

# Runs kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 23s]
val_accuracy: 0.7265306115150452

Best val_accuracy So Far: 0.7287463545799255
Total elapsed time: 00h 25m 20s


In [15]:
# Gets top 3 model hyperparameters and prints the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 16, 'num_layers': 4, 'units_0': 26, 'units_1': 26, 'units_2': 6, 'units_3': 6, 'units_4': 21, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0015'}
{'activation': 'relu', 'first_units': 16, 'num_layers': 3, 'units_0': 11, 'units_1': 26, 'units_2': 16, 'units_3': 1, 'units_4': 26, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0045'}
{'activation': 'relu', 'first_units': 16, 'num_layers': 5, 'units_0': 26, 'units_1': 6, 'units_2': 6, 'units_3': 16, 'units_4': 21, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0023'}


In [16]:
# Defines the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 11
hidden_nodes_layer2 = 26

# Creates model instance
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Checks the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 11)                495       
                                                                 
 dense_4 (Dense)             (None, 26)                312       
                                                                 
 dense_5 (Dense)             (None, 1)                 27        
                                                                 
Total params: 834 (3.26 KB)
Trainable params: 834 (3.26 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Compiles the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Creates a call back that saves the model's weights every five epoches
weight_callback = ModelCheckpoint(
                                  filepath='AlphabetSoupCharityWeights.h5',
                                  save_weights_only=True,
                                  period=5
                                )

# Trains the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=weight_callback)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5946 - accuracy: 0.6994
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5601 - accuracy: 0.7252
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5558 - accuracy: 0.7285
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5533 - accuracy: 0.7296
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5516 - accuracy: 0.7304
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7320
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7319
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7328
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5479 - accuracy: 0.7332
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5475 - accura

In [19]:
# Evaluates the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5533 - accuracy: 0.7244 - 555ms/epoch - 2ms/step
Loss: 0.5532683730125427, Accuracy: 0.7244315147399902


In [20]:
# Exports the model to HDF5 file
nn.save('AlphabetSoupCharity.h5')
files.download("AlphabetSoupCharity.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>